# M-BERT Baseline Modeling

### Appendix: Evaluating different checkpoints on validation set
Here, I evaluate the checkpoints from a previous run which did not print out results as training progressed

In [ ]:
# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = "../models",
    save_strategy               = "epoch",
    evaluation_strategy         = "epoch"
)

#Define Trainer object
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = val_set_dataset,
    compute_metrics = compute_metrics
)

#Start pre-training!
trainer.train()

In [ ]:
from glob import glob

#List to store validation set results for the different checkpoints
val_results = []

#Iterate through all the subfolders in the main directory          
for folder in glob('../models/*/'):
    
    #If it is a model save checkpoint
    if 'checkpoint' in folder:
    
        #Load the checkpoint
        model = BertForSequenceClassification.from_pretrained(folder)

        training_args = TrainingArguments(
        output_dir = "../model_predictions",
        do_predict = True)

        trainer = Trainer(
            model           = model,
            args            = training_args,
            compute_metrics =compute_metrics)

        model_preds_and_results = trainer.predict(val_set_dataset)

        val_results.append(model_preds_and_results[-1])

In [ ]:
import re
results_df = pd.DataFrame(val_results)
results_df.insert(0,'model_checkpoint',[re.search('checkpoint-\d+',checkpoint)[0] for checkpoint in (glob('../models/*/')) if 'checkpoint' in checkpoint])

In [94]:
results_df.sort_values('test_accuracy')

,model_checkpoint,test_loss,test_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_steps_per_second
1,checkpoint-29,0.985939,0.565789,0.408890,0.320118,0.565789,90.8152,0.837,0.110
4,checkpoint-116,1.327372,0.644737,0.570270,0.660313,0.644737,86.2949,0.881,0.116
3,checkpoint-58,0.918521,0.657895,0.635467,0.652943,0.657895,86.1463,0.882,0.116
8,checkpoint-203,1.201182,0.684211,0.658236,0.646460,0.684211,86.9010,0.875,0.115
2,checkpoint-290,1.361241,0.697368,0.686283,0.682873,0.697368,88.4328,0.859,0.113
9,checkpoint-261,1.327470,0.697368,0.677505,0.660383,0.697368,86.7420,0.876,0.115
6,checkpoint-145,1.171518,0.710526,0.684230,0.684649,0.710526,85.9271,0.884,0.116
7,checkpoint-232,1.224277,0.710526,0.682639,0.669507,0.710526,83.8298,0.907,0.119
0,checkpoint-87,0.884741,0.723684,0.695574,0.674425,0.723684,121.4265,0.626,0.082
5,checkpoint-174,0.905348,0.789474,0.756738,0.727052,0.789474,85.8604,0.885,0.116


Checkpoint-174 appeared to do best across most of the metrics